In [7]:
import nltk
from nltk.corpus import stopwords
import re
import numpy as np
from pprint import pprint
import logging
import texthero as hero
from texthero import preprocessing
import pandas as pd
import tqdm
from IPython.display import Audio, display
def allDone():
    display(Audio(url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav', autoplay=True))
import warnings
warnings.filterwarnings("ignore")
import csv
csv.field_size_limit()
131072
csv.field_size_limit(256 << 10)
131072
csv.field_size_limit()
262144
logging.basicConfig(
    format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
warnings.filterwarnings("ignore", category=DeprecationWarning)
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.test.utils import common_corpus, common_dictionary
from gensim.models import LdaModel
from pprint import pprint
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
import numpy as np
import gensim
import matplotlib.pyplot as plt
import operator
import re
import os
from IPython.core.display import display, HTML
import warnings
#import pyLDAvis.gensim_models
from sklearn.manifold import TSNE
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
import gensim.downloader as api
from gensim.models import CoherenceModel

In [17]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'lda_model' is not defined

In [47]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [48]:
def get_tune(j):
    #%% Load file
    df = pd.read_csv(f'C:/Users/user/Documents/1. GitHub/Projeto 5-NLP-SP-City-Hall/tables_clean_lem/{j}.csv', sep=r'Γ')
    df = df.dropna()
    train_text = df['clean_lem']
    #%% Create tokens
    texts = [p.lower().split() for p in train_text]
    id2word = gensim.corpora.Dictionary(texts)
    corpus = [id2word.doc2bow(text) for text in texts]
    # print(texts[:1])
    # train_text[300]
    # id2word[16]
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=30,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)
    
    # Compute Coherence Score
    #coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
    #coherence_lda = coherence_model_lda.get_coherence()
    #print('\nCoherence Score: ', coherence_lda)
    
    grid = {}
    grid['Validation_Set'] = {}
    # Topics range
    min_topics = 2
    max_topics = 50
    step_size = 1
    topics_range = range(min_topics, max_topics, step_size)
    # Alpha parameter
    alpha = list(np.arange(0.01, 1, 0.3))
    alpha.append('symmetric')
    alpha.append('asymmetric')
    # Beta parameter
    beta = list(np.arange(0.01, 1, 0.3))
    beta.append('symmetric')
    # Validation sets
    num_of_docs = len(corpus)
    corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
                   # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
                   gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
                   corpus]
    corpus_title = ['75% Corpus', '100% Corpus']
    model_results = {'Validation_Set': [],
                     'Topics': [],
                     'Alpha': [],
                     'Beta': [],
                     'Coherence': []
                    }
    # Can take a long time to run
    if 1 == 1:
        pbar = tqdm.tqdm(total=540)
    
        # iterate through validation corpuses
        for i in range(len(corpus_sets)):
            # iterate through number of topics
            for k in topics_range:
                # iterate through alpha values
                for a in alpha:
                    # iterare through beta values
                    for b in beta:
                        # get the coherence score for the given parameters
                        cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                      k=k, a=a, b=b)
                        # Save the model results
                        model_results['Validation_Set'].append(corpus_title[i])
                        model_results['Topics'].append(k)
                        model_results['Alpha'].append(a)
                        model_results['Beta'].append(b)
                        model_results['Coherence'].append(cv)
                    
                        pbar.update(1)
        pd.DataFrame(model_results).to_csv(f'{j}.csv', index=False)
        pbar.close()
        allDone()

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [49]:
get_tune("direitos_humanos_doria_covas")

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)





  0%|                                                                                          | 0/540 [00:00<?, ?it/s]

NameError: name 'texts' is not defined

In [33]:
df['clean_lem']

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'df' is not defined

In [60]:
a = pd.read_csv('lda_tuning_results.csv')
a

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Validation_Set,Topics,Alpha,Beta,Coherence
0,75% Corpus,2,0.01,0.01,0.355302
1,75% Corpus,2,0.01,0.31,0.381825
2,75% Corpus,2,0.01,0.61,0.371778
3,75% Corpus,2,0.01,0.9099999999999999,0.371778
4,75% Corpus,2,0.01,symmetric,0.380559
5,75% Corpus,2,0.31,0.01,0.369021
6,75% Corpus,2,0.31,0.31,0.362637
7,75% Corpus,2,0.31,0.61,0.370759
8,75% Corpus,2,0.31,0.9099999999999999,0.379165
9,75% Corpus,2,0.31,symmetric,0.380559


In [61]:
pd.set_option('display.max_rows', None)
a.sort_values(by=['Coherence'], ascending=False)

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Validation_Set,Topics,Alpha,Beta,Coherence
388,75% Corpus,14,asymmetric,0.9099999999999999,0.537070
1852,100% Corpus,15,symmetric,0.61,0.531259
267,75% Corpus,10,asymmetric,0.61,0.531051
207,75% Corpus,8,asymmetric,0.61,0.530856
623,75% Corpus,22,symmetric,0.9099999999999999,0.528328
603,75% Corpus,22,0.01,0.9099999999999999,0.527852
723,75% Corpus,26,0.01,0.9099999999999999,0.527827
533,75% Corpus,19,symmetric,0.9099999999999999,0.520929
1953,100% Corpus,19,0.01,0.9099999999999999,0.520135
202,75% Corpus,8,symmetric,0.61,0.519248


In [4]:
P=int(input()) #população total da cidade
TxContagio=float(input()) #taxa de contagio
Z= 1022 #pessoas contaminadas no dia zero
IC= 0.79*P #Numero de pessoas infectadas necessario pra haver imunidade de rebanho

DIA=2
U= Z*TxContagio #novas pessoas contaminadas ontem
D= (U*TxContagio)+Z+U #total de pessoas contaminadas hoje
while D<IC:
  Z = Z + U 
  U=U*(TxContagio)
  D=U*(1+TxContagio)+Z
  DIA+=1

print("A cidade conseguiu imunidade coletiva em",DIA,"dias")

1000000
2
A cidade conseguiu imunidade coletiva em 7 dias
